<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/Cnn_bilstm_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
!ls /content/drive/MyDrive/

'carte visite badra.gdoc'
 Classroom
'Colab Notebooks'
 django
 entreprise-1-20220417T102515Z-001.zip
'Espace citoyen.pdf'
 ez-learning
 maghrebi
'model (1).pth'
 preprocessed_tweets.csv
 pre_tweets.csv
 Projet_Java_FediBoussaada_LaribiSkander_2A
 Projet_Microservices
 trainer2.pth
'Untitled presentation.gslides'


In [56]:
import pandas as pd
filtered_df = pd.read_csv('/content/drive/MyDrive/pre_tweets.csv')
filtered_df.head()

,Unnamed: 0,id,tweets,dialect
0,0,1009754958479151232,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...,1
1,1,1009794751548313600,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...,1
2,2,1019989115490787200,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...,1
3,3,1035479791758135168,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,1
4,4,1035481122921164800,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...,1


In [57]:
filtered_df['tweets'][1848]

'دخيل روحك الحلوه ه حبيت الفيسات بيلبقالك ملك ونص دخيل قلبكم واله يعطيكم العافيه احلي فريق قصيخولي غمارالغانم خمسهونص'

In [58]:
filtered_df = filtered_df.dropna()

In [59]:
# create boolean mask of NaN values
nan_mask = filtered_df.isna()

# sum NaN values
nan_sum = nan_mask.sum().sum()

print(nan_sum)  # output: 3

0


In [60]:
filtered_df['dialect'].value_counts()

2    36494
3    36483
0    36483
1    36478
Name: dialect, dtype: int64

In [61]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [62]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)

In [63]:
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# use the stopwords
stop_words = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens

X = remove_common_words(X)


X = pad_sequences(X, maxlen=max_length)

In [65]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [66]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape , y_val.shape

((118209, 200),
 (14594, 200),
 (13135, 200),
 (118209, 4),
 (14594, 4),
 (13135, 4))

In [67]:
# Convert data to PyTorch tensors
X_train, y_train = torch.tensor(X_train), torch.tensor(y_train)
X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [68]:
class DialectDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [69]:
# Create dataloaders for training, validation, and test sets
train_dataset = DialectDataset(X_train, y_train)
val_dataset = DialectDataset(X_val, y_val)
test_dataset = DialectDataset(X_test, y_test)

In [70]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [71]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
# from sklearn.metrics import accuracy_score
# from tqdm import tqdm
# import numpy as np

# vocab_size = 20000
# embedding_dim = 100
# hidden_dim = 128
# output_dim = 4
# num_layers = 2
# bidirectional = True
# lr = 0.01
# batch_size = 64
# num_epochs = 100
# patience = 5

# class BiLSTMAttention(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
#         super(BiLSTMAttention, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3),stride =(2,2), padding=(1, 0))
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3),stride =(2,2), padding=(1, 0))
#         self.lstm = nn.LSTM(1600, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=0.5, batch_first=True)
#         self.fc = nn.Linear(hidden_dim * 2, output_dim)
#         self.dropout = nn.Dropout(0.5)
#         self.attention = SelfAttention(hidden_dim * 2)

#     def forward(self, x):
#         embedded = self.embedding(x)
#         conv1_out = F.relu(self.conv1(embedded.unsqueeze(1)))
#         conv2_out = F.relu(self.conv2(conv1_out))
#         conv2_out = conv2_out.permute(0, 2, 1, 3)
#         conv2_out = conv2_out.reshape(conv2_out.size(0), conv2_out.size(1), -1)
#         lstm_in = conv2_out.permute(0, 2, 1)
#         lstm_out, _ = self.lstm(lstm_in)
#         lstm_out = self.dropout(lstm_out)
#         attention_out , attention_weights= self.attention(lstm_out)
#         fc_out = self.fc(attention_out)
#         return fc_out, attention_weights

# class SelfAttention(nn.Module):
#     def __init__(self, hidden_dim):
#         super(SelfAttention, self).__init__()
#         self.projection = nn.Sequential(
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.Tanh(),
#             nn.Linear(hidden_dim, 1)
#         )
        
#     def forward(self, encoder_outputs):
#         energy = self.projection(encoder_outputs)
#         weights = F.softmax(energy.flatten(start_dim=1), dim=1)
#         outputs = (encoder_outputs * weights.unsqueeze(-1)).sum(dim=1)
#         return outputs, weights

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np

input_dim = 50
vocab_size = 20000
embedding_dim = 100
hidden_dim = 1
output_dim = 4
num_layers = 2
bidirectional = True
lr = 0.01
batch_size = 64
num_epochs = 100
patience = 3

class ConvLSTMWithSelfAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, bidirectional=True):
        super(ConvLSTMWithSelfAttention, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3,3), stride=(2,2), padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), stride=(2,2), padding=1)
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        
        self.self_attention = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.Tanh(),
            nn.Linear(hidden_dim*2, 1),
            nn.Softmax(dim=1)
        )
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
    def forward(self, x):
        # Convolutional layers
        x = x.unsqueeze(0) # Add channel dimension
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), x.size(1), -1)
        
        # LSTM layers
        h0 = torch.zeros(self.num_layers * self.hidden_dim * 2 if self.bidirectional else self.num_layers * self.hidden_dim, x.size(0), self.hidden_dim).to("cuda")
        c0 = torch.zeros(self.num_layers * self.hidden_dim * 2 if self.bidirectional else self.num_layers * self.hidden_dim, x.size(0), self.hidden_dim).to("cuda")
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Self-attention
        attention_weights = self.self_attention(lstm_out)
        attention_weights = attention_weights.permute(0, 2, 1)
        weighted_out = torch.bmm(attention_weights, lstm_out)
        out = torch.sum(weighted_out, dim=1)
        
        # Fully-connected layer
        out = self.fc(out)

        # Apply sigmoid function to output tensor and round to nearest integer
        # out = torch.sigmoid(out).round()

        return out

In [83]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ConvLSTMWithSelfAttention(input_dim, hidden_dim, output_dim, num_layers, bidirectional=True).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)
criterion = nn.BCEWithLogitsLoss()
counter = 0
for epoch in tqdm(range(24)):
     model.train()
     running_loss = 0.0
     with torch.cuda.device(0):
       for batch in train_dataloader:
         inputs, labels = batch[0].to(device), batch[1].to(device)
         inputs = inputs.float()  # Convert input tensor to float
         optimizer.zero_grad()
         labels = labels.float()
         outputs = model(inputs)
         #print(outputs.shape)
         if labels.shape != outputs.shape:
           print(f'Shape of outputs: {outputs.shape}, shape of labels: {labels.shape}, Labels: {labels}')
           counter +=1
           print(counter)
         else:
           loss = criterion(outputs, labels)
           loss.backward()
           optimizer.step()
           running_loss += loss.item()
           preds = torch.sigmoid(outputs) > 0.5
         epoch_loss = running_loss / (len(train_dataloader))

     best_val_loss = np.inf
     patience = 3
     counter = 0
     val_loss = 0.0
     model.eval()
     with torch.no_grad():
         for batch in val_dataloader:
             inputs, labels = batch[0].to(device), batch[1].to(device)
             inputs = inputs.float()  # Convert input tensor to float
             labels = labels.float()
             outputs = model(inputs)
             if labels.shape != outputs.shape:
               print(f'Shape of outputs: {outputs.shape}, shape of labels: {labels.shape}, Labels: {labels}')
             else:
               loss = criterion(outputs, labels)
               val_loss += loss.item()
               val_preds = torch.sigmoid(outputs) > 0.5
               acc = accuracy_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy())
     val_loss = val_loss / (len(val_dataloader))



     # Check if the validation loss has improved
     if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
     else:
         counter += 1

     # Stop the training process if the validation loss hasn't improved for `patience` epochs
     if counter >= patience:
         break
     print(" epoch loss :", epoch_loss , "| val loss :", val_loss)
     print('Train Accuracy: {:.4f}'.format(acc))

print("Training stopped after epoch", epoch)
        

  0%|          | 0/24 [00:00<?, ?it/s]

Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([1, 4]), Labels: tensor([[0., 0., 0., 1.]], device='cuda:0')
1


  4%|▍         | 1/24 [00:08<03:24,  8.89s/it]

Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([15, 4]), Labels: tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]], device='cuda:0')
 epoch loss : 0.563924860096339 | val loss : 0.5596324216972277
Train Accuracy: 0.0000
Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([1, 4]), Labels: tensor([[1., 0., 0., 0.]], device='cuda:0')
1


  8%|▊         | 2/24 [00:16<03:03,  8.33s/it]

Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([15, 4]), Labels: tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]], device='cuda:0')
 epoch loss : 0.5623890424703623 | val loss : 0.5596849924152337
Train Accuracy: 0.0000
Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([1, 4]), Labels: tensor([[1., 0., 0., 0.]], device='cuda:0')
1


 12%|█▎        | 3/24 [00:25<03:00,  8.60s/it]

Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([15, 4]), Labels: tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]], device='cuda:0')
 epoch loss : 0.5622740395012356 | val loss : 0.5597434587848996
Train Accuracy: 0.0000


 12%|█▎        | 3/24 [00:33<03:57, 11.30s/it]


KeyboardInterrupt: ignored

In [73]:
from sklearn.metrics import f1_score, precision_score, recall_score
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        labels = labels.float()
        inputs = inputs.float()
        outputs= model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        if labels.shape != outputs.shape:
               print(f'Shape of outputs: {outputs.shape}, shape of labels: {labels.shape}, Labels: {labels}')
        else:
          y_true.extend(labels.cpu().detach().numpy())
          y_pred.extend(preds.cpu().detach().numpy())
test_acc = accuracy_score(y_true, y_pred)
test_f1 = f1_score(y_true, y_pred, average='macro')
test_pres = precision_score(y_true, y_pred, average='macro')
test_recall = recall_score(y_true, y_pred, average='macro')
print('Test Accuracy: {:.4f}'.format(test_acc))
print('Test f1 score: {:.4f}'.format(test_f1))
print('Test precision: {:.4f}'.format(test_pres))
print('Test recall: {:.4f}'.format(test_recall))

Shape of outputs: torch.Size([64, 4]), shape of labels: torch.Size([2, 4]), Labels: tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.]], device='cuda:0')
Test Accuracy: 0.0094
Test f1 score: 0.1969
Test precision: 0.1232
Test recall: 0.4895


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
# Set the model to evaluation mode
model.eval()
device = "cuda"
# Tokenize the new complaint and pad the sequence
#new_complaint = ['واش دير للعشى']
#new_complaint  = ["شوكران علا هاد "]
#new_complaint  = ["نبي نروح للحوش"]
new_complaint  = ["شبيك شتحب "]
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=max_length)

# Convert the padded sequence to a PyTorch tensor and move it to the device (e.g., GPU) if available
padded_tensor = torch.LongTensor(padded).to(device)

# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred = model(padded_tensor)

# Move the prediction back to the CPU and convert to a numpy array
#pred = pred.cpu().numpy()

# Map the prediction to a class label using the CLASS_DICT{'DZ': 0, 'LY': 1, 'MA': 2, 'TN': 3}
CLASS_DICT = {0: "DZ",1: "LY", 2: "MA",3: "TN"}
class_label = CLASS_DICT[np.argmax(pred[0].cpu().numpy(), axis=1)[0]]

# Print the prediction and the predicted class label
print(class_label)

RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), 'model.pth')